In [6]:
%pylab inline
import numpy as np
import torch as th
import torch.nn as nn
import torch.optim as optim
import sklearn.model_selection as selection
import sklearn.metrics as metrics
import sklearn.linear_model as linear
from sklearn import datasets
from sklearn.decomposition import PCA
from scipy.stats import ttest_rel
from sklearn import tree

# import some data to play with
# This is a mutli-class classification problem
iris = datasets.load_iris()
X = iris.data 
y = iris.target

Populating the interactive namespace from numpy and matplotlib


# Machine Learning Pipelines

In this notebook, we will explore the different facilities that sklearn provides to help us build simple machine learning pipelines. We will also perform a simple machine learning experiment on a simple dataset

## Training-Testing Set

Recall that in the training-training set methodology, we simply split the dataset into two (unequally) sized blocks - a training set and a testing set. 

The training set is used to derive the model and its paramaters, and the testing set is then used to assess the quality of the trained model.

In [9]:
# let's split our data into a training set and a testing set
# We want 20% of our data to be used for testing and the rest for training
X_train, X_test, y_train, y_test = selection.train_test_split(X, y, test_size=0.2, shuffle=True)

In [16]:
# instantiate blank models
lr_model = linear.LogisticRegression(solver='lbfgs', multi_class='auto')

In [17]:
# train models
lr_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Know we need to run predictions of the trained models on the testing set to get a sense of how well the model performs

In [20]:
pred_lr = lr_model.predict(X_test) # get predicted class

In [21]:
print('F1-micro score of LR is ', metrics.f1_score(y_test, pred_lr, average='micro'))
print('F1-macro score of LR is ', metrics.f1_score(y_test, pred_lr, average='macro'))

F1-micro score of LR is  1.0
F1-macro score of LR is  1.0


In [23]:
print('Confusion Matrix of LR is')
print(metrics.confusion_matrix(y_test, pred_lr))

Confusion Matrix of LR is
[[ 6  0  0]
 [ 0 11  0]
 [ 0  0 13]]


We may have just lucked out with our split, let's run a k-fold with 5 folds

In [41]:
kf = selection.StratifiedKFold(n_splits=5) # We used Stratified KFold since we have multiple classes

In [42]:
kf.get_n_splits(X)

5

In [43]:
f1_micro = []
f1_macro= []
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = linear.LogisticRegression(multi_class='auto', solver='saga')
    lr.fit(X_train, y_train)
    predictions = lr.predict(X_test)
    f1_micro.append(metrics.f1_score(y_test, predictions, average='micro'))
    f1_macro.append(metrics.f1_score(y_test, predictions, average='macro'))
    
f1_micro = np.array(f1_micro)
f1_macro = np.array(f1_macro)

/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/inzamamrahaman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag

In [40]:
print('Average f1 micro is ', np.mean(f1_micro), ' with std. dev ', np.std(f1_micro))
print('Average f1 macro is ', np.mean(f1_macro), ' with std. dev ', np.std(f1_macro))

Average f1 micro is  0.9866666666666667  with std. dev  0.016329931618554516
Average f1 macro is  0.9866332497911445  with std. dev  0.01637085876546819
